In [1]:
#import neccessary libraries we would be working with
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
import graphviz
import pickle
import random
import warnings
warnings.filterwarnings('ignore')

### **Populate Data And Load Model**

In [4]:
# Get data: This CSV holds match and team statistics for international games
!cp /content/drive/MyDrive/international_matches.csv -d /content

# list of participating countries in the 2022 FIFA World Cup
FIFA_2022_TEAMS = ['Qatar', 'Brazil', 'Belgium', 'France', 'Argentina', 'England', 'Spain', 'Portugal', 'Mexico',
'Netherlands', 'Denmark', 'Germany', 'Uruguay', 'Switzerland', 'USA','Croatia', 'Senegal',
'IR Iran', 'Japan', 'Morocco', 'Serbia', 'Poland', 'Korea Republic', 'Tunisia', 'Cameroon', 
'Canada', 'Ecuador','Saudi Arabia', 'Ghana', 'Wales','Costa Rica', 'Australia']
FIFA_2022_TEAMS.sort() # sort by alphabetical order for uniformity

model = pickle.load(open('/content/drive/MyDrive/Finalized_2022_World_Cup_Model.sav', 'rb')) # load the model weights

df = pd.read_csv('/content/international_matches.csv', parse_dates = ['date']) # read csv into dataframe in again
df_2022 = df[df['date'].dt.year >= 2022]  # take only matches played in 2022 so as to get the most recent scores for each team

FIFA_2022_LATEST_RANKING = [3,38,2,1,43,41,31,12,10,44,5,4,11,61,20,24,28,13,22,8,26,9,50,51,18,21,7,15,30,16,14,19] # in same order as in FIFA_2022_TEAMS.sort()
features = ['fifa_rank', 'goalkeeper_score', 'defense_score', 'offense_score', 'midfield_score'] # these are the features in which we would populate our statistics dataframe with

# create dataframe
FIFA_2022_TEAMS_dict = {}
for f in features:
  FIFA_2022_TEAMS_dict[f] = {}
  for team in FIFA_2022_TEAMS:
    FIFA_2022_TEAMS_dict[f][team] = []

d = {'team': FIFA_2022_TEAMS, 'goalkeeper_score': [0]*32, 'defense_score': [0]*32,'offense_score': [0]*32,          
'midfield_score': [0]*32, 'fifa_rank': [0]*32 }
FIFA_2022_ratings = pd.DataFrame(data=d)

for i, t in enumerate(FIFA_2022_TEAMS):
    df_team_home = df_2022[df_2022['home_team'] == t]
    df_team_away = df_2022[df_2022['away_team'] == t]

    FIFA_2022_ratings['goalkeeper_score'].iloc[i] = (df_team_home['home_team_goalkeeper_score'].mean() + df_team_away['away_team_goalkeeper_score'].mean())/2 
    FIFA_2022_ratings['defense_score'].iloc[i] = (df_team_home['home_team_mean_defense_score'].mean() + df_team_away['away_team_mean_defense_score'].mean())/2 
    FIFA_2022_ratings['offense_score'].iloc[i] = (df_team_home['home_team_mean_offense_score'].mean() + df_team_away['away_team_mean_offense_score'].mean())/2 
    FIFA_2022_ratings['midfield_score'].iloc[i] = (df_team_home['home_team_mean_midfield_score'].mean() + df_team_away['away_team_mean_midfield_score'].mean())/2 
    FIFA_2022_ratings['fifa_rank'].iloc[i] = FIFA_2022_LATEST_RANKING[i]

# these data were still missing after populating the data, so I had to manuallt fill them in
# Qatar ratings
FIFA_2022_ratings['goalkeeper_score'].iloc[22] = 66
FIFA_2022_ratings['defense_score'].iloc[22] = 68
FIFA_2022_ratings['offense_score'].iloc[22] = 71
FIFA_2022_ratings['midfield_score'].iloc[22] = 70

# Tunisia goalkeeper ranking
FIFA_2022_ratings['goalkeeper_score'].iloc[28] = 71.5
FIFA_2022_ratings

,team,goalkeeper_score,defense_score,offense_score,midfield_score,fifa_rank
0,Argentina,84.0,82.2,89.0,84.0,3
1,Australia,77.0,72.0,72.3,73.5,38
2,Belgium,89.0,80.8,85.7,85.5,2
3,Brazil,89.0,84.8,86.3,85.5,1
4,Cameroon,67.0,76.8,77.7,75.0,43
5,Canada,76.0,69.2,73.0,78.0,41
6,Costa Rica,88.0,71.5,70.3,69.2,31
7,Croatia,82.0,77.8,76.7,84.2,12
8,Denmark,85.0,80.2,77.7,80.2,10
9,Ecuador,71.0,73.5,76.0,74.5,44


### **Round of 16**

In [5]:
round_of_16 = [['Netherlands', 'USA'], ['Argentina', 'Australia'], ['Japan', 'Croatia'], ['Brazil', 'Korea Republic'], ['England', 'Senegal'], ['France', 'Poland'], ['Morocco', 'Spain'], ['Switzerland', 'Portugal']]

round_of_16_winners = []
draws = []
for match in round_of_16:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # team_1 wins
    round_of_16_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # team_1 and team_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # team_1 loses ie. team_2 wins 
    round_of_16_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Netherlands VS USA
MATCH OUTCOME Netherlands VS USA WINNER: Netherlands

MATCH PLAYED Argentina VS Australia
MATCH OUTCOME Argentina VS Australia WINNER: Argentina

MATCH PLAYED Japan VS Croatia
MATCH OUTCOME Japan VS Croatia WINNER: Croatia

MATCH PLAYED Brazil VS Korea Republic
MATCH OUTCOME Brazil VS Korea Republic WINNER: Brazil

MATCH PLAYED England VS Senegal
MATCH OUTCOME England VS Senegal WINNER: England

MATCH PLAYED France VS Poland
MATCH OUTCOME France VS Poland WINNER: France

MATCH PLAYED Morocco VS Spain
MATCH OUTCOME Morocco VS Spain WINNER: Spain

MATCH PLAYED Switzerland VS Portugal
MATCH OUTCOME Switzerland VS Portugal WINNER: Portugal



### **Quarter-Finals**

In [6]:
quarter_finals = [['Netherlands', 'Argentina'], ['Croatia', 'Brazil'], ['England', 'France'], ['Spain', 'Portugal']]

quarter_final_winners = []
draws = []
for match in quarter_finals:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # team_1 wins
    quarter_final_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # team_1 and team_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # team_1 loses ie. team_2 wins 
    quarter_final_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Netherlands VS Argentina
MATCH OUTCOME Netherlands VS Argentina WINNER: Argentina

MATCH PLAYED Croatia VS Brazil
MATCH OUTCOME Croatia VS Brazil DRAW

MATCH PLAYED England VS France
MATCH OUTCOME England VS France WINNER: France

MATCH PLAYED Spain VS Portugal
MATCH OUTCOME Spain VS Portugal DRAW



### **Semi-Finals**

As the model can only predict win, draw, or lose and cannot predict the outcome of penalty shootouts we run multiple permations as draws were predicted in the quarter-finals

In [7]:
semi_finals = [['Argentina', 'Brazil'], ['France', 'Spain']]

semi_final_winners = []
draws = []
for match in semi_finals:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # team_1 wins
    semi_final_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # team_1 and team_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # team_1 loses ie. team_2 wins 
    semi_final_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Argentina VS Brazil
MATCH OUTCOME Argentina VS Brazil DRAW

MATCH PLAYED France VS Spain
MATCH OUTCOME France VS Spain DRAW



In [9]:
semi_finals = [['Argentina', 'Brazil'], ['France', 'Portugal']]

semi_final_winners = []
draws = []
for match in semi_finals:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # team_1 wins
    semi_final_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # team_1 and team_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # team_1 loses ie. team_2 wins 
    semi_final_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Argentina VS Brazil
MATCH OUTCOME Argentina VS Brazil DRAW

MATCH PLAYED France VS Portugal
MATCH OUTCOME France VS Portugal DRAW



In [8]:
semi_finals = [['Argentina', 'Croatia'], ['France', 'Spain']]

semi_final_winners = []
draws = []
for match in semi_finals:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # team_1 wins
    semi_final_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # team_1 and team_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # team_1 loses ie. team_2 wins 
    semi_final_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Argentina VS Croatia
MATCH OUTCOME Argentina VS Croatia WINNER: Argentina

MATCH PLAYED France VS Spain
MATCH OUTCOME France VS Spain DRAW



In [10]:
semi_finals = [['Argentina', 'Croatia'], ['France', 'Portugal']]

semi_final_winners = []
draws = []
for match in semi_finals:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # team_1 wins
    semi_final_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # team_1 and team_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # team_1 loses ie. team_2 wins 
    semi_final_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Argentina VS Croatia
MATCH OUTCOME Argentina VS Croatia WINNER: Argentina

MATCH PLAYED France VS Portugal
MATCH OUTCOME France VS Portugal DRAW



Despite there being a few permutaions, I decidd to go with historic performance on penalties to determine who would progress to the finals. 

Between Croatia and Brazil, Brazil should win on penalties. meanwhile, betweeb Spain and Portugal, Spain should win on penalties. 

### **Finals And Third Place**
As draws were predicted once more, we have no definite prediction on who would progress to the finals

In [ ]:
MATCH PLAYED Argentina VS Brazil
MATCH OUTCOME Argentina VS Brazil DRAW
==================================

==================================
MATCH PLAYED France VS Spain
MATCH OUTCOME France VS Spain DRAW

#### Permutation 1

In [11]:
third_place = [['Argentina', 'Spain']]

third_place_winners = []
draws = []
for match in third_place:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # team_1 wins
    third_place_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # team_1 and team_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # team_1 loses ie. team_2 wins 
    third_place_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Argentina VS Spain
MATCH OUTCOME Argentina VS Spain DRAW



In [12]:
finals = [['France', 'Brazil']]

final_winners = []
draws = []
for match in finals:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # is team_1 wins
    final_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # is both team_1 and eam_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # is both team_1 loses ie. team_2 wins 
    final_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED France VS Brazil
MATCH OUTCOME France VS Brazil WINNER: France



#### Permutaion 2

In [13]:
third_place = [['Argentina', 'France']]

third_place_winners = []
draws = []
for match in third_place:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # team_1 wins
    third_place_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # team_1 and team_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # team_1 loses ie. team_2 wins 
    third_place_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Argentina VS France
MATCH OUTCOME Argentina VS France WINNER: France



In [14]:
finals = [['Spain', 'Brazil']]

final_winners = []
draws = []
for match in finals:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # is team_1 wins
    final_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # is both team_1 and eam_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # is both team_1 loses ie. team_2 wins 
    final_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Spain VS Brazil
MATCH OUTCOME Spain VS Brazil DRAW



#### Permutation 3

In [15]:
third_place = [['Brazil', 'France']]

third_place_winners = []
draws = []
for match in third_place:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # team_1 wins
    third_place_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # team_1 and team_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # team_1 loses ie. team_2 wins 
    third_place_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Brazil VS France
MATCH OUTCOME Brazil VS France WINNER: France



In [16]:
finals = [['Argentina', 'Spain']]

final_winners = []
draws = []
for match in finals:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # is team_1 wins
    final_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # is both team_1 and eam_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # is both team_1 loses ie. team_2 wins 
    final_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Argentina VS Spain
MATCH OUTCOME Argentina VS Spain DRAW



#### Permutation 4

In [17]:
third_place = [['Brazil', 'Spain']]

third_place_winners = []
draws = []
for match in third_place:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # team_1 wins
    third_place_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # team_1 and team_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # team_1 loses ie. team_2 wins 
    third_place_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Brazil VS Spain
MATCH OUTCOME Brazil VS Spain WINNER: Spain



In [18]:
finals = [['Argentina', 'France']]

final_winners = []
draws = []
for match in finals:
  team_1 = match[0]
  team_2 = match[1]
  
  print("==================================")

  print("MATCH PLAYED", team_1, "VS", team_2)
  
  data = [float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['fifa_rank']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['fifa_rank']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['goalkeeper_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['goalkeeper_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['defense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['offense_score']), \
          float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_1]['midfield_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['defense_score']), \
            float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['offense_score']), float(FIFA_2022_ratings[FIFA_2022_ratings['team'] == team_2]['midfield_score']) ]
  data = [data]

  prediction = model.predict(data)[0]

  if prediction == 2: # is team_1 wins
    final_winners.append(team_1)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_1)
  elif prediction == 1: # is both team_1 and eam_2 draws
    draws.append((team_1, team_2))
    print("MATCH OUTCOME", team_1, "VS", team_2, "DRAW")
  else: # is both team_1 loses ie. team_2 wins 
    final_winners.append(team_2)
    print("MATCH OUTCOME", team_1, "VS", team_2, "WINNER:", team_2)

  print("==================================")
  print()

MATCH PLAYED Argentina VS France
MATCH OUTCOME Argentina VS France WINNER: France



## **Conclusion**
While there is no definite prediction, it is fairly safe to say France are the favourites to win the competition